In [1]:
# =========================================
# VitalDB Dataset Analysis and Exploration
# =========================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("🏥 VitalDB Dataset Analysis")
print("=" * 50)

# Set up data directory
data_dir = "/Users/nguyennghia/EHR/DATA/vital_files_subsets/"
print(f"📁 Data directory: {data_dir}")

# List all CSV files
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print(f"📊 Found {len(csv_files)} CSV files:")
for i, file in enumerate(csv_files, 1):
    print(f"   {i}. {file}")

print(f"\n🔍 Available .vital files: {len(os.listdir(os.path.join(data_dir, 'vital_files_subsets')))}")


🏥 VitalDB Dataset Analysis
📁 Data directory: /Users/nguyennghia/EHR/DATA/vital_files_subsets/
📊 Found 5 CSV files:
   1. lab_parameters.csv
   2. track_names.csv
   3. clinical_parameters.csv
   4. lab_data.csv
   5. clinical_data.csv

🔍 Available .vital files: 10


In [2]:
# =========================================
# 1. Load and Analyze Each CSV File
# =========================================

def analyze_csv_file(file_path, file_name):
    """
    Comprehensive analysis of a CSV file
    """
    print(f"\n📄 ANALYZING: {file_name}")
    print("=" * 60)
    
    try:
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Basic information
        print(f"📊 Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
        print(f"💾 Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
        
        # Column information
        print(f"\n📋 COLUMNS ({len(df.columns)}):")
        for i, col in enumerate(df.columns, 1):
            dtype = df[col].dtype
            non_null = df[col].count()
            null_count = df[col].isnull().sum()
            null_pct = (null_count / len(df)) * 100
            
            print(f"   {i:2d}. {col:<30} | {str(dtype):<12} | {non_null:>6,} non-null | {null_pct:>5.1f}% missing")
        
        # Data types summary
        print(f"\n🔢 DATA TYPES:")
        dtype_counts = df.dtypes.value_counts()
        for dtype, count in dtype_counts.items():
            print(f"   • {dtype}: {count} columns")
        
        # Missing values summary
        missing_summary = df.isnull().sum()
        missing_cols = missing_summary[missing_summary > 0].sort_values(ascending=False)
        
        if len(missing_cols) > 0:
            print(f"\n❌ MISSING VALUES ({len(missing_cols)} columns with missing data):")
            for col, missing_count in missing_cols.head(10).items():
                missing_pct = (missing_count / len(df)) * 100
                print(f"   • {col:<30}: {missing_count:>6,} ({missing_pct:>5.1f}%)")
        else:
            print(f"\n✅ NO MISSING VALUES!")
        
        # Sample data
        print(f"\n📝 SAMPLE DATA (first 3 rows):")
        print(df.head(3).to_string())
        
        # Unique values for categorical columns
        categorical_cols = df.select_dtypes(include=['object']).columns
        if len(categorical_cols) > 0:
            print(f"\n🏷️  CATEGORICAL COLUMNS:")
            for col in categorical_cols[:5]:  # Show first 5 categorical columns
                unique_count = df[col].nunique()
                print(f"   • {col}: {unique_count:,} unique values")
                if unique_count <= 20:  # Show unique values if not too many
                    unique_vals = df[col].value_counts().head(10)
                    print(f"     Top values: {dict(unique_vals)}")
        
        return df
        
    except Exception as e:
        print(f"❌ Error loading {file_name}: {str(e)}")
        return None

# Analyze all CSV files
csv_data = {}
for csv_file in csv_files:
    file_path = os.path.join(data_dir, csv_file)
    df = analyze_csv_file(file_path, csv_file)
    if df is not None:
        csv_data[csv_file] = df



📄 ANALYZING: lab_parameters.csv
📊 Shape: 33 rows × 5 columns
💾 Memory usage: 0.01 MB

📋 COLUMNS (5):
    1. Parameter                      | object       |     33 non-null |   0.0% missing
    2. Category                       | object       |     33 non-null |   0.0% missing
    3. Description                    | object       |     33 non-null |   0.0% missing
    4. Unit                           | object       |     32 non-null |   3.0% missing
    5. Reference value                | object       |     33 non-null |   0.0% missing

🔢 DATA TYPES:
   • object: 5 columns

❌ MISSING VALUES (1 columns with missing data):
   • Unit                          :      1 (  3.0%)

📝 SAMPLE DATA (first 3 rows):
  Parameter Category             Description       Unit Reference value
0       wbc      CBC  White blood cell count  ×1000/mcL            4~10
1        hb      CBC              Hemoglobin       g/dL           13~17
2       hct      CBC              Hematocrit          %           39~52

In [3]:
# =========================================
# 2. Detailed Analysis of Each Dataset
# =========================================

def detailed_dataset_summary(df, dataset_name):
    """
    Provide detailed summary and explanation for each dataset
    """
    print(f"\n🔬 DETAILED ANALYSIS: {dataset_name}")
    print("=" * 70)
    
    # Dataset overview
    print(f"📊 Dataset Overview:")
    print(f"   • Total Records: {len(df):,}")
    print(f"   • Total Features: {len(df.columns)}")
    print(f"   • Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Column analysis
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    datetime_cols = df.select_dtypes(include=['datetime64']).columns
    
    print(f"\n📋 Column Types:")
    print(f"   • Numeric: {len(numeric_cols)}")
    print(f"   • Categorical: {len(categorical_cols)}")
    print(f"   • DateTime: {len(datetime_cols)}")
    
    # Statistical summary for numeric columns
    if len(numeric_cols) > 0:
        print(f"\n📈 Numeric Columns Statistics:")
        numeric_stats = df[numeric_cols].describe()
        print(numeric_stats.round(2).to_string())
    
    # Unique values analysis
    print(f"\n🔍 Unique Values Analysis:")
    for col in df.columns:
        unique_count = df[col].nunique()
        total_count = len(df)
        uniqueness_ratio = unique_count / total_count
        
        print(f"   • {col:<30}: {unique_count:>6,} unique ({uniqueness_ratio:>5.1%} of total)")
        
        # Show sample values for categorical columns with few unique values
        if col in categorical_cols and unique_count <= 10:
            unique_vals = df[col].value_counts()
            print(f"     Values: {dict(unique_vals)}")
    
    return {
        'shape': df.shape,
        'columns': list(df.columns),
        'numeric_cols': list(numeric_cols),
        'categorical_cols': list(categorical_cols),
        'datetime_cols': list(datetime_cols),
        'missing_summary': df.isnull().sum().to_dict(),
        'dtypes': df.dtypes.to_dict()
    }

# Analyze each dataset in detail
dataset_summaries = {}
for file_name, df in csv_data.items():
    summary = detailed_dataset_summary(df, file_name)
    dataset_summaries[file_name] = summary



🔬 DETAILED ANALYSIS: lab_parameters.csv
📊 Dataset Overview:
   • Total Records: 33
   • Total Features: 5
   • Memory Usage: 0.01 MB

📋 Column Types:
   • Numeric: 0
   • Categorical: 5
   • DateTime: 0

🔍 Unique Values Analysis:
   • Parameter                     :     33 unique (100.0% of total)
   • Category                      :      4 unique (12.1% of total)
     Values: {'Chemistry': 17, 'ABGA': 6, 'CBC': 5, 'Coagulation': 5}
   • Description                   :     33 unique (100.0% of total)
   • Unit                          :     13 unique (39.4% of total)
   • Reference value               :     32 unique (97.0% of total)

🔬 DETAILED ANALYSIS: track_names.csv
📊 Dataset Overview:
   • Total Records: 196
   • Total Features: 4
   • Memory Usage: 0.05 MB

📋 Column Types:
   • Numeric: 0
   • Categorical: 4
   • DateTime: 0

🔍 Unique Values Analysis:
   • Parameter                     :    196 unique (100.0% of total)
   • Description                   :    173 unique (88.3% o

In [4]:
# =========================================
# 3. Dataset Explanations and Interpretations
# =========================================

print("🏥 VitalDB Dataset Explanations")
print("=" * 70)

# Clinical Data Analysis
if 'clinical_data.csv' in csv_data:
    clinical_df = csv_data['clinical_data.csv']
    print(f"\n📋 CLINICAL DATA (clinical_data.csv)")
    print("─" * 50)
    print("This dataset contains patient demographic and clinical information.")
    print("Key information:")
    
    # Check for common clinical columns
    clinical_cols = clinical_df.columns.tolist()
    print(f"   • Total columns: {len(clinical_cols)}")
    
    # Look for patient identifiers
    id_cols = [col for col in clinical_cols if any(keyword in col.lower() for keyword in ['id', 'patient', 'case'])]
    if id_cols:
        print(f"   • Patient ID columns: {id_cols}")
    
    # Look for demographic info
    demo_cols = [col for col in clinical_cols if any(keyword in col.lower() for keyword in ['age', 'sex', 'gender', 'height', 'weight', 'bmi'])]
    if demo_cols:
        print(f"   • Demographic columns: {demo_cols}")
    
    # Look for outcome variables
    outcome_cols = [col for col in clinical_cols if any(keyword in col.lower() for keyword in ['outcome', 'mortality', 'death', 'survival', 'los'])]
    if outcome_cols:
        print(f"   • Outcome columns: {outcome_cols}")
    
    print(f"   • Sample data shape: {clinical_df.shape}")
    if len(clinical_df) > 0:
        print(f"   • First few column names: {clinical_cols[:10]}")

# Clinical Parameters Analysis
if 'clinical_parameters.csv' in csv_data:
    params_df = csv_data['clinical_parameters.csv']
    print(f"\n⚙️ CLINICAL PARAMETERS (clinical_parameters.csv)")
    print("─" * 50)
    print("This dataset contains definitions and metadata for clinical parameters.")
    print("Key information:")
    
    params_cols = params_df.columns.tolist()
    print(f"   • Total columns: {len(params_cols)}")
    
    # Look for parameter definitions
    def_cols = [col for col in params_cols if any(keyword in col.lower() for keyword in ['name', 'description', 'unit', 'range'])]
    if def_cols:
        print(f"   • Parameter definition columns: {def_cols}")
    
    print(f"   • Sample data shape: {params_df.shape}")

# Lab Data Analysis
if 'lab_data.csv' in csv_data:
    lab_df = csv_data['lab_data.csv']
    print(f"\n🧪 LAB DATA (lab_data.csv)")
    print("─" * 50)
    print("This dataset contains laboratory test results and measurements.")
    print("Key information:")
    
    lab_cols = lab_df.columns.tolist()
    print(f"   • Total columns: {len(lab_cols)}")
    
    # Look for time-related columns
    time_cols = [col for col in lab_cols if any(keyword in col.lower() for keyword in ['time', 'date', 'timestamp'])]
    if time_cols:
        print(f"   • Time-related columns: {time_cols}")
    
    # Look for value columns
    value_cols = [col for col in lab_cols if any(keyword in col.lower() for keyword in ['value', 'result', 'measurement'])]
    if value_cols:
        print(f"   • Value columns: {value_cols}")
    
    print(f"   • Sample data shape: {lab_df.shape}")

# Lab Parameters Analysis
if 'lab_parameters.csv' in csv_data:
    lab_params_df = csv_data['lab_parameters.csv']
    print(f"\n🔬 LAB PARAMETERS (lab_parameters.csv)")
    print("─" * 50)
    print("This dataset contains definitions and metadata for laboratory parameters.")
    print("Key information:")
    
    lab_params_cols = lab_params_df.columns.tolist()
    print(f"   • Total columns: {len(lab_params_cols)}")
    print(f"   • Sample data shape: {lab_params_df.shape}")

# Track Names Analysis
if 'track_names.csv' in csv_data:
    tracks_df = csv_data['track_names.csv']
    print(f"\n🎯 TRACK NAMES (track_names.csv)")
    print("─" * 50)
    print("This dataset contains names and identifiers for vital sign tracks.")
    print("Key information:")
    
    track_cols = tracks_df.columns.tolist()
    print(f"   • Total columns: {len(track_cols)}")
    print(f"   • Sample data shape: {tracks_df.shape}")
    
    # Look for track identifiers
    track_id_cols = [col for col in track_cols if any(keyword in col.lower() for keyword in ['track', 'id', 'name', 'code'])]
    if track_id_cols:
        print(f"   • Track identifier columns: {track_id_cols}")


🏥 VitalDB Dataset Explanations

📋 CLINICAL DATA (clinical_data.csv)
──────────────────────────────────────────────────
This dataset contains patient demographic and clinical information.
Key information:
   • Total columns: 74
   • Patient ID columns: ['caseid', 'subjectid', 'casestart', 'caseend', 'intraop_crystalloid', 'intraop_colloid']
   • Demographic columns: ['age', 'sex', 'height', 'weight', 'bmi']
   • Outcome columns: ['death_inhosp']
   • Sample data shape: (6388, 74)
   • First few column names: ['caseid', 'subjectid', 'casestart', 'caseend', 'anestart', 'aneend', 'opstart', 'opend', 'adm', 'dis']

⚙️ CLINICAL PARAMETERS (clinical_parameters.csv)
──────────────────────────────────────────────────
This dataset contains definitions and metadata for clinical parameters.
Key information:
   • Total columns: 4
   • Parameter definition columns: ['Description', 'Unit']
   • Sample data shape: (81, 4)

🧪 LAB DATA (lab_data.csv)
──────────────────────────────────────────────────
Th

In [5]:
# =========================================
# 4. Data Quality Assessment
# =========================================

print("\n🔍 DATA QUALITY ASSESSMENT")
print("=" * 70)

def assess_data_quality(df, dataset_name):
    """
    Comprehensive data quality assessment
    """
    print(f"\n📊 Quality Assessment: {dataset_name}")
    print("─" * 50)
    
    total_rows = len(df)
    total_cols = len(df.columns)
    
    # Completeness
    missing_data = df.isnull().sum()
    total_missing = missing_data.sum()
    completeness = ((total_rows * total_cols - total_missing) / (total_rows * total_cols)) * 100
    
    print(f"📈 Completeness: {completeness:.1f}%")
    print(f"   • Total cells: {total_rows * total_cols:,}")
    print(f"   • Missing cells: {total_missing:,}")
    print(f"   • Complete cells: {(total_rows * total_cols - total_missing):,}")
    
    # Columns with missing data
    cols_with_missing = missing_data[missing_data > 0]
    if len(cols_with_missing) > 0:
        print(f"\n❌ Columns with missing data ({len(cols_with_missing)}):")
        for col, missing_count in cols_with_missing.head(10).items():
            missing_pct = (missing_count / total_rows) * 100
            print(f"   • {col:<30}: {missing_count:>6,} ({missing_pct:>5.1f}%)")
    
    # Duplicates
    duplicate_rows = df.duplicated().sum()
    duplicate_pct = (duplicate_rows / total_rows) * 100
    print(f"\n🔄 Duplicates: {duplicate_rows:,} rows ({duplicate_pct:.1f}%)")
    
    # Data types consistency
    print(f"\n🔢 Data Types:")
    dtype_counts = df.dtypes.value_counts()
    for dtype, count in dtype_counts.items():
        print(f"   • {dtype}: {count} columns")
    
    # Potential data quality issues
    issues = []
    
    # Check for completely empty columns
    empty_cols = df.columns[df.isnull().all()].tolist()
    if empty_cols:
        issues.append(f"Empty columns: {empty_cols}")
    
    # Check for columns with very high missing rates
    high_missing_cols = missing_data[missing_data > total_rows * 0.5].index.tolist()
    if high_missing_cols:
        issues.append(f"High missing rate (>50%): {high_missing_cols}")
    
    # Check for potential ID columns with duplicates
    potential_id_cols = [col for col in df.columns if 'id' in col.lower()]
    for col in potential_id_cols:
        if df[col].nunique() != len(df):
            issues.append(f"Non-unique ID column: {col}")
    
    if issues:
        print(f"\n⚠️  Potential Issues:")
        for issue in issues:
            print(f"   • {issue}")
    else:
        print(f"\n✅ No major data quality issues detected!")
    
    return {
        'completeness': completeness,
        'missing_cells': total_missing,
        'duplicate_rows': duplicate_rows,
        'empty_columns': empty_cols,
        'high_missing_columns': high_missing_cols
    }

# Assess quality for each dataset
quality_summaries = {}
for file_name, df in csv_data.items():
    quality_summary = assess_data_quality(df, file_name)
    quality_summaries[file_name] = quality_summary



🔍 DATA QUALITY ASSESSMENT

📊 Quality Assessment: lab_parameters.csv
──────────────────────────────────────────────────
📈 Completeness: 99.4%
   • Total cells: 165
   • Missing cells: 1
   • Complete cells: 164

❌ Columns with missing data (1):
   • Unit                          :      1 (  3.0%)

🔄 Duplicates: 0 rows (0.0%)

🔢 Data Types:
   • object: 5 columns

✅ No major data quality issues detected!

📊 Quality Assessment: track_names.csv
──────────────────────────────────────────────────
📈 Completeness: 100.0%
   • Total cells: 784
   • Missing cells: 0
   • Complete cells: 784

🔄 Duplicates: 0 rows (0.0%)

🔢 Data Types:
   • object: 4 columns

✅ No major data quality issues detected!

📊 Quality Assessment: clinical_parameters.csv
──────────────────────────────────────────────────
📈 Completeness: 91.4%
   • Total cells: 324
   • Missing cells: 28
   • Complete cells: 296

❌ Columns with missing data (1):
   • Unit                          :     28 ( 34.6%)

🔄 Duplicates: 0 rows (0.

In [6]:
# =========================================
# 5. Comprehensive Summary and Recommendations
# =========================================

print("\n📋 COMPREHENSIVE SUMMARY")
print("=" * 70)

print("🏥 VitalDB Dataset Overview:")
print("─" * 50)

total_datasets = len(csv_data)
total_records = sum(len(df) for df in csv_data.values())
total_features = sum(len(df.columns) for df in csv_data.values())

print(f"📊 Dataset Statistics:")
print(f"   • Total CSV files analyzed: {total_datasets}")
print(f"   • Total records across all datasets: {total_records:,}")
print(f"   • Total features across all datasets: {total_features}")

print(f"\n📁 Dataset Breakdown:")
for file_name, df in csv_data.items():
    print(f"   • {file_name:<25}: {len(df):>6,} records × {len(df.columns):>3} columns")

print(f"\n🎯 Dataset Purposes:")
print(f"   • clinical_data.csv: Patient demographics and clinical information")
print(f"   • clinical_parameters.csv: Metadata for clinical parameters")
print(f"   • lab_data.csv: Laboratory test results and measurements")
print(f"   • lab_parameters.csv: Metadata for laboratory parameters")
print(f"   • track_names.csv: Vital sign track identifiers and names")

print(f"\n🔍 Key Findings:")

# Overall data quality
overall_completeness = np.mean([qs['completeness'] for qs in quality_summaries.values()])
total_missing = sum(qs['missing_cells'] for qs in quality_summaries.values())
total_duplicates = sum(qs['duplicate_rows'] for qs in quality_summaries.values())

print(f"   • Overall data completeness: {overall_completeness:.1f}%")
print(f"   • Total missing values across all datasets: {total_missing:,}")
print(f"   • Total duplicate records: {total_duplicates:,}")

# Data type distribution
all_dtypes = {}
for summary in dataset_summaries.values():
    for dtype, count in summary['dtypes'].items():
        dtype_str = str(count)
        all_dtypes[dtype_str] = all_dtypes.get(dtype_str, 0) + 1

print(f"\n📊 Data Type Distribution:")
for dtype, count in all_dtypes.items():
    print(f"   • {dtype}: {count} columns")

print(f"\n💡 Recommendations for Analysis:")
print(f"   1. 🏥 Clinical Data Integration:")
print(f"      - Merge clinical_data.csv with lab_data.csv using patient IDs")
print(f"      - Use clinical_parameters.csv for parameter interpretation")
print(f"      - Apply track_names.csv for vital sign identification")

print(f"\n   2. 🧪 Time-Series Analysis:")
print(f"      - Focus on lab_data.csv for temporal patterns")
print(f"      - Handle missing values using time-series imputation methods")
print(f"      - Consider forward-fill for continuous monitoring data")

print(f"\n   3. 📈 Feature Engineering:")
print(f"      - Extract temporal features (trends, variability)")
print(f"      - Create derived clinical scores and ratios")
print(f"      - Normalize values using lab_parameters.csv ranges")

print(f"\n   4. 🔍 Data Quality Improvements:")
print(f"      - Address missing values systematically")
print(f"      - Validate clinical ranges using parameter definitions")
print(f"      - Remove or impute outliers based on medical knowledge")

print(f"\n   5. 🎯 Predictive Modeling:")
print(f"      - Use clinical_data.csv for patient outcomes")
print(f"      - Create time-series features from lab_data.csv")
print(f"      - Apply transformer models for sequence prediction")

print(f"\n🚀 Next Steps:")
print(f"   1. Load and merge datasets using appropriate keys")
print(f"   2. Implement time-series preprocessing pipeline")
print(f"   3. Apply missing value imputation strategies")
print(f"   4. Create predictive models for clinical outcomes")
print(f"   5. Validate results using medical domain knowledge")

print(f"\n" + "="*70)
print(f"✅ VitalDB Dataset Analysis Complete!")
print(f"📊 Ready for advanced time-series modeling and clinical prediction tasks")



📋 COMPREHENSIVE SUMMARY
🏥 VitalDB Dataset Overview:
──────────────────────────────────────────────────
📊 Dataset Statistics:
   • Total CSV files analyzed: 5
   • Total records across all datasets: 935,146
   • Total features across all datasets: 91

📁 Dataset Breakdown:
   • lab_parameters.csv       :     33 records ×   5 columns
   • track_names.csv          :    196 records ×   4 columns
   • clinical_parameters.csv  :     81 records ×   4 columns
   • lab_data.csv             : 928,448 records ×   4 columns
   • clinical_data.csv        :  6,388 records ×  74 columns

🎯 Dataset Purposes:
   • clinical_data.csv: Patient demographics and clinical information
   • clinical_parameters.csv: Metadata for clinical parameters
   • lab_data.csv: Laboratory test results and measurements
   • lab_parameters.csv: Metadata for laboratory parameters
   • track_names.csv: Vital sign track identifiers and names

🔍 Key Findings:
   • Overall data completeness: 94.5%
   • Total missing values acros

In [8]:
# =========================================
# 7. Clinical Data Integration
# =========================================

print("🔗 CLINICAL DATA INTEGRATION")
print("=" * 70)

# Load the datasets for integration
clinical_df = csv_data['clinical_data.csv']
lab_df = csv_data['lab_data.csv']
clinical_params_df = csv_data['clinical_parameters.csv']
lab_params_df = csv_data['lab_parameters.csv']
tracks_df = csv_data['track_names.csv']

print(f"📊 Initial Dataset Sizes:")
print(f"   • Clinical Data: {clinical_df.shape[0]:,} patients × {clinical_df.shape[1]} features")
print(f"   • Lab Data: {lab_df.shape[0]:,} lab records × {lab_df.shape[1]} features")
print(f"   • Clinical Parameters: {clinical_params_df.shape[0]:,} parameters")
print(f"   • Lab Parameters: {lab_params_df.shape[0]:,} lab parameters")
print(f"   • Track Names: {tracks_df.shape[0]:,} vital sign tracks")

# Explore the relationship between datasets
print(f"\n🔍 Exploring Dataset Relationships:")

# Check caseid overlap between clinical and lab data
clinical_caseids = set(clinical_df['caseid'].unique())
lab_caseids = set(lab_df['caseid'].unique())
common_caseids = clinical_caseids.intersection(lab_caseids)

print(f"   • Unique caseids in clinical_data: {len(clinical_caseids):,}")
print(f"   • Unique caseids in lab_data: {len(lab_caseids):,}")
print(f"   • Common caseids: {len(common_caseids):,}")
print(f"   • Coverage: {len(common_caseids)/len(clinical_caseids)*100:.1f}% of clinical cases have lab data")

# Check lab parameter names vs lab data names
lab_param_names = set(lab_params_df['Parameter'].unique())
lab_data_names = set(lab_df['name'].unique())
common_lab_names = lab_param_names.intersection(lab_data_names)

print(f"   • Lab parameters defined: {len(lab_param_names)}")
print(f"   • Lab data parameter names: {len(lab_data_names)}")
print(f"   • Matched lab parameters: {len(common_lab_names)}")
print(f"   • Lab parameter coverage: {len(common_lab_names)/len(lab_data_names)*100:.1f}%")

# Show sample of lab parameters
print(f"\n📋 Sample Lab Parameters:")
sample_lab_params = lab_params_df.head(10)
for idx, row in sample_lab_params.iterrows():
    print(f"   • {row['Parameter']:<8}: {row['Description']} ({row['Unit']})")

print(f"\n📋 Sample Lab Data:")
sample_lab_data = lab_df.head(10)
print(sample_lab_data.to_string())

print(f"\n📋 Sample Clinical Data:")
sample_clinical = clinical_df[['caseid', 'subjectid', 'age', 'sex', 'height', 'weight', 'death_inhosp']].head(5)
print(sample_clinical.to_string())


🔗 CLINICAL DATA INTEGRATION
📊 Initial Dataset Sizes:
   • Clinical Data: 6,388 patients × 74 features
   • Lab Data: 928,448 lab records × 4 features
   • Clinical Parameters: 81 parameters
   • Lab Parameters: 33 lab parameters
   • Track Names: 196 vital sign tracks

🔍 Exploring Dataset Relationships:
   • Unique caseids in clinical_data: 6,388
   • Unique caseids in lab_data: 5,796
   • Common caseids: 5,796
   • Coverage: 90.7% of clinical cases have lab data
   • Lab parameters defined: 33
   • Lab data parameter names: 34
   • Matched lab parameters: 33
   • Lab parameter coverage: 97.1%

📋 Sample Lab Parameters:
   • wbc     : White blood cell count (×1000/mcL)
   • hb      : Hemoglobin (g/dL)
   • hct     : Hematocrit (%)
   • plt     : Platelet count (×1000/mcL)
   • esr     : Erythrocyte sedimentation rate (mm/hr)
   • gluc    : Glucose (mg/dL)
   • tprot   : Total protein (g/dL)
   • alb     : Albumin (g/dL)
   • tbil    : Total bilirubin (mg/dL)
   • ast     : Asparate tran

In [9]:
# =========================================
# 8. Lab Data Pivoting and Time-Series Preparation
# =========================================

print("📊 LAB DATA PIVOTING AND TIME-SERIES PREPARATION")
print("=" * 70)

# Create lab parameter mapping for better interpretation
lab_param_mapping = lab_params_df.set_index('Parameter').to_dict('index')
print(f"📋 Lab Parameter Mapping Created:")
print(f"   • {len(lab_param_mapping)} parameters mapped with descriptions and units")

# Show sample mappings
print(f"\n📋 Sample Parameter Mappings:")
for param in list(lab_param_mapping.keys())[:5]:
    mapping = lab_param_mapping[param]
    print(f"   • {param}: {mapping.get('Description', 'N/A')} ({mapping.get('Unit', 'N/A')})")

# Analyze lab data structure
print(f"\n🔍 Lab Data Structure Analysis:")
print(f"   • Total lab records: {len(lab_df):,}")
print(f"   • Time range: {lab_df['dt'].min():,} to {lab_df['dt'].max():,}")
print(f"   • Unique parameters: {lab_df['name'].nunique()}")
print(f"   • Unique cases: {lab_df['caseid'].nunique()}")

# Check time distribution
print(f"\n⏰ Time Distribution Analysis:")
time_stats = lab_df['dt'].describe()
print(f"   • Mean time: {time_stats['mean']:,.0f}")
print(f"   • Median time: {time_stats['50%']:,.0f}")
print(f"   • Min time: {time_stats['min']:,.0f}")
print(f"   • Max time: {time_stats['max']:,.0f}")

# Check parameter frequency
print(f"\n📊 Parameter Frequency Analysis:")
param_counts = lab_df['name'].value_counts()
print(f"   • Most common parameters:")
for param, count in param_counts.head(10).items():
    pct = (count / len(lab_df)) * 100
    print(f"     {param:<8}: {count:>6,} records ({pct:>4.1f}%)")

# Create time-series pivot table for a sample of patients
print(f"\n🔄 Creating Time-Series Pivot Table...")

# Sample first 100 patients for demonstration
sample_caseids = list(common_caseids)[:100]
sample_lab_data = lab_df[lab_df['caseid'].isin(sample_caseids)].copy()

print(f"   • Sample size: {len(sample_caseids)} patients")
print(f"   • Sample lab records: {len(sample_lab_data):,}")

# Create pivot table with caseid, dt as index and parameters as columns
print(f"\n📊 Pivoting lab data to time-series format...")
pivot_lab_data = sample_lab_data.pivot_table(
    index=['caseid', 'dt'], 
    columns='name', 
    values='result', 
    aggfunc='mean'  # Use mean if multiple values for same time point
)

print(f"   • Pivot table shape: {pivot_lab_data.shape}")
print(f"   • Time points per patient (avg): {len(pivot_lab_data) / len(sample_caseids):.1f}")

# Show sample of pivoted data
print(f"\n📋 Sample Pivoted Lab Data:")
print(pivot_lab_data.head(10).to_string())

# Analyze missing patterns in pivoted data
print(f"\n❌ Missing Data Analysis in Pivoted Format:")
missing_summary = pivot_lab_data.isnull().sum().sort_values(ascending=False)
print(f"   • Parameters with missing data:")
for param, missing_count in missing_summary.head(10).items():
    missing_pct = (missing_count / len(pivot_lab_data)) * 100
    print(f"     {param:<8}: {missing_count:>6,} missing ({missing_pct:>5.1f}%)")

print(f"   • Overall completeness: {(1 - pivot_lab_data.isnull().sum().sum() / (pivot_lab_data.shape[0] * pivot_lab_data.shape[1])) * 100:.1f}%")


📊 LAB DATA PIVOTING AND TIME-SERIES PREPARATION
📋 Lab Parameter Mapping Created:
   • 33 parameters mapped with descriptions and units

📋 Sample Parameter Mappings:
   • wbc: White blood cell count (×1000/mcL)
   • hb: Hemoglobin (g/dL)
   • hct: Hematocrit (%)
   • plt: Platelet count (×1000/mcL)
   • esr: Erythrocyte sedimentation rate (mm/hr)

🔍 Lab Data Structure Analysis:
   • Total lab records: 928,448
   • Time range: -7,775,687 to 7,775,588
   • Unique parameters: 34
   • Unique cases: 5796

⏰ Time Distribution Analysis:
   • Mean time: 233,410
   • Median time: 70,157
   • Min time: -7,775,687
   • Max time: 7,775,588

📊 Parameter Frequency Analysis:
   • Most common parameters:
     hct     : 58,498 records ( 6.3%)
     k       : 55,486 records ( 6.0%)
     na      : 55,463 records ( 6.0%)
     hb      : 43,976 records ( 4.7%)
     wbc     : 43,941 records ( 4.7%)
     plt     : 43,529 records ( 4.7%)
     cl      : 40,863 records ( 4.4%)
     cr      : 37,311 records ( 4.0%)

In [10]:
# =========================================
# 9. Clinical Data Integration and Merging
# =========================================

print("🔗 CLINICAL DATA INTEGRATION AND MERGING")
print("=" * 70)

# Merge clinical data with lab data summary statistics
print("📊 Creating Integrated Clinical Dataset...")

# Create lab data summary for each patient
print("   1. Creating lab data summary statistics per patient...")

lab_summary_stats = lab_df.groupby('caseid').agg({
    'dt': ['count', 'min', 'max'],  # Number of measurements, time range
    'result': ['mean', 'std', 'min', 'max', 'count']  # Statistical summary
}).round(3)

# Flatten column names
lab_summary_stats.columns = ['_'.join(col).strip() for col in lab_summary_stats.columns]
lab_summary_stats = lab_summary_stats.reset_index()

print(f"   • Lab summary shape: {lab_summary_stats.shape}")

# Create parameter-specific summaries for key lab values
print("   2. Creating parameter-specific summaries...")

key_parameters = ['wbc', 'hb', 'hct', 'plt', 'na', 'k', 'gluc', 'alb', 'cr', 'bun']
param_summaries = []

for param in key_parameters:
    if param in lab_df['name'].values:
        param_data = lab_df[lab_df['name'] == param].groupby('caseid')['result'].agg([
            'count', 'mean', 'std', 'min', 'max'
        ]).round(3)
        
        param_data.columns = [f'{param}_{col}' for col in param_data.columns]
        param_data = param_data.reset_index()
        param_summaries.append(param_data)
        
        print(f"     • {param}: {len(param_data)} patients with data")

# Merge parameter summaries
if param_summaries:
    param_summary_df = param_summaries[0]
    for df in param_summaries[1:]:
        param_summary_df = param_summary_df.merge(df, on='caseid', how='outer')
    
    print(f"   • Parameter summary shape: {param_summary_df.shape}")
else:
    param_summary_df = pd.DataFrame({'caseid': clinical_df['caseid']})

# Merge clinical data with lab summaries
print("   3. Merging clinical data with lab summaries...")

integrated_clinical = clinical_df.merge(lab_summary_stats, on='caseid', how='left')
integrated_clinical = integrated_clinical.merge(param_summary_df, on='caseid', how='left')

print(f"   • Integrated clinical dataset shape: {integrated_clinical.shape}")
print(f"   • Original clinical features: {clinical_df.shape[1]}")
print(f"   • Added lab summary features: {integrated_clinical.shape[1] - clinical_df.shape[1]}")

# Analyze integration results
print(f"\n📊 Integration Analysis:")
print(f"   • Patients with lab data: {integrated_clinical['dt_count'].notna().sum():,}")
print(f"   • Patients without lab data: {integrated_clinical['dt_count'].isna().sum():,}")
print(f"   • Integration coverage: {integrated_clinical['dt_count'].notna().sum() / len(integrated_clinical) * 100:.1f}%")

# Show sample of integrated data
print(f"\n📋 Sample Integrated Clinical Data:")
sample_cols = ['caseid', 'age', 'sex', 'death_inhosp', 'dt_count', 'result_mean', 'wbc_mean', 'hb_mean', 'na_mean']
available_cols = [col for col in sample_cols if col in integrated_clinical.columns]
print(integrated_clinical[available_cols].head(10).to_string())

# Create final integrated dataset for analysis
print(f"\n✅ Final Integrated Dataset Created!")
print(f"   • Total patients: {len(integrated_clinical):,}")
print(f"   • Total features: {integrated_clinical.shape[1]}")
print(f"   • Memory usage: {integrated_clinical.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Save integrated dataset for further analysis
integrated_clinical.to_csv('/Users/nguyennghia/EHR/DATA/vital_files_subsets/integrated_clinical_data.csv', index=False)
print(f"   • Saved to: integrated_clinical_data.csv")


🔗 CLINICAL DATA INTEGRATION AND MERGING
📊 Creating Integrated Clinical Dataset...
   1. Creating lab data summary statistics per patient...
   • Lab summary shape: (5796, 9)
   2. Creating parameter-specific summaries...
     • wbc: 5371 patients with data
     • hb: 5371 patients with data
     • hct: 5457 patients with data
     • plt: 5369 patients with data
     • na: 5402 patients with data
     • k: 5402 patients with data
     • gluc: 5091 patients with data
     • alb: 5278 patients with data
     • cr: 5170 patients with data
     • bun: 5170 patients with data
   • Parameter summary shape: (5634, 51)
   3. Merging clinical data with lab summaries...
   • Integrated clinical dataset shape: (6388, 132)
   • Original clinical features: 74
   • Added lab summary features: 58

📊 Integration Analysis:
   • Patients with lab data: 5,796
   • Patients without lab data: 592
   • Integration coverage: 90.7%

📋 Sample Integrated Clinical Data:
   caseid age sex  death_inhosp  dt_count  

In [11]:
# =========================================
# 10. Time-Series Data Preparation for Deep Learning
# =========================================

print("⏰ TIME-SERIES DATA PREPARATION FOR DEEP LEARNING")
print("=" * 70)

# Create time-series dataset for transformer models
print("📊 Creating Time-Series Dataset for Deep Learning...")

# Select patients with sufficient lab data
min_lab_records = 10  # Minimum number of lab records per patient
patient_lab_counts = lab_df.groupby('caseid').size()
eligible_patients = patient_lab_counts[patient_lab_counts >= min_lab_records].index

print(f"   • Patients with ≥{min_lab_records} lab records: {len(eligible_patients):,}")
print(f"   • Eligible patients coverage: {len(eligible_patients)/len(clinical_caseids)*100:.1f}%")

# Filter lab data for eligible patients
eligible_lab_data = lab_df[lab_df['caseid'].isin(eligible_patients)].copy()

# Create time-series sequences
print("🔄 Creating Time-Series Sequences...")

def create_time_series_sequences(lab_data, clinical_data, max_length=48):
    """
    Create time-series sequences for each patient
    """
    sequences = []
    targets = []
    patient_info = []
    
    for caseid in eligible_patients[:100]:  # Limit for demonstration
        # Get lab data for this patient
        patient_lab = lab_data[lab_data['caseid'] == caseid].copy()
        
        if len(patient_lab) < 5:  # Skip patients with too few records
            continue
            
        # Sort by time
        patient_lab = patient_lab.sort_values('dt')
        
        # Get clinical outcome
        clinical_info = clinical_data[clinical_data['caseid'] == caseid]
        if len(clinical_info) == 0:
            continue
            
        target = clinical_info['death_inhosp'].iloc[0]
        
        # Create sequences of fixed length
        for i in range(0, len(patient_lab) - max_length + 1, max_length // 2):
            sequence_lab = patient_lab.iloc[i:i + max_length]
            
            # Pivot to get parameter columns
            seq_pivot = sequence_lab.pivot_table(
                index='dt', 
                columns='name', 
                values='result', 
                aggfunc='mean'
            )
            
            # Fill missing values with forward fill
            seq_pivot = seq_pivot.fillna(method='ffill').fillna(method='bfill')
            
            # Ensure consistent column structure
            if seq_pivot.shape[0] < max_length:
                # Pad with last values if sequence is too short
                padding_needed = max_length - seq_pivot.shape[0]
                last_row = seq_pivot.iloc[-1:].copy()
                for _ in range(padding_needed):
                    seq_pivot = pd.concat([seq_pivot, last_row])
            
            # Take only the required length
            seq_pivot = seq_pivot.iloc[:max_length]
            
            sequences.append(seq_pivot.values)
            targets.append(target)
            patient_info.append({
                'caseid': caseid,
                'sequence_length': len(sequence_lab),
                'time_range': (sequence_lab['dt'].min(), sequence_lab['dt'].max())
            })
    
    return sequences, targets, patient_info

# Create sequences
print("   • Creating sequences for first 100 eligible patients...")
sequences, targets, patient_info = create_time_series_sequences(eligible_lab_data, clinical_df)

print(f"   • Total sequences created: {len(sequences)}")
print(f"   • Sequence shape: {sequences[0].shape if sequences else 'No sequences'}")
print(f"   • Target distribution: {pd.Series(targets).value_counts().to_dict()}")

# Create parameter mapping for sequence columns
if sequences:
    # Get parameter names from the first sequence
    param_names = eligible_lab_data['name'].unique()
    param_names = sorted(param_names)  # Sort for consistency
    
    print(f"   • Parameters in sequences: {len(param_names)}")
    print(f"   • Sample parameters: {param_names[:10]}")
    
    # Create parameter info for interpretation
    sequence_param_info = []
    for param in param_names:
        if param in lab_param_mapping:
            info = lab_param_mapping[param]
            sequence_param_info.append({
                'parameter': param,
                'description': info.get('Description', 'N/A'),
                'unit': info.get('Unit', 'N/A'),
                'category': info.get('Category', 'N/A')
            })
        else:
            sequence_param_info.append({
                'parameter': param,
                'description': 'Unknown',
                'unit': 'Unknown',
                'category': 'Unknown'
            })
    
    param_info_df = pd.DataFrame(sequence_param_info)
    print(f"   • Parameter info dataframe: {param_info_df.shape}")

# Analyze sequence characteristics
if sequences:
    print(f"\n📊 Sequence Analysis:")
    seq_lengths = [len(seq) for seq in sequences]
    print(f"   • Sequence lengths: min={min(seq_lengths)}, max={max(seq_lengths)}, mean={np.mean(seq_lengths):.1f}")
    
    # Check for missing values in sequences
    total_values = sum(seq.size for seq in sequences)
    missing_values = sum(np.isnan(seq).sum() for seq in sequences)
    print(f"   • Missing values: {missing_values:,} / {total_values:,} ({missing_values/total_values*100:.1f}%)")
    
    # Show sample sequence
    print(f"\n📋 Sample Sequence (Patient {patient_info[0]['caseid']}):")
    sample_seq = sequences[0]
    sample_df = pd.DataFrame(sample_seq[:10], columns=param_names[:sample_seq.shape[1]])
    print(sample_df.round(2).to_string())

print(f"\n✅ Time-Series Dataset Preparation Complete!")
print(f"   • Ready for transformer model training")
print(f"   • Sequences can be used for mortality prediction")
print(f"   • Parameter interpretation available through lab_param_mapping")


⏰ TIME-SERIES DATA PREPARATION FOR DEEP LEARNING
📊 Creating Time-Series Dataset for Deep Learning...
   • Patients with ≥10 lab records: 5,412
   • Eligible patients coverage: 84.7%
🔄 Creating Time-Series Sequences...
   • Creating sequences for first 100 eligible patients...
   • Total sequences created: 543
   • Sequence shape: (48, 17)
   • Target distribution: {0: 543}
   • Parameters in sequences: 34
   • Sample parameters: ['alb', 'alt', 'ammo', 'aptt', 'ast', 'be', 'bun', 'ccr', 'cl', 'cr']
   • Parameter info dataframe: (34, 4)

📊 Sequence Analysis:
   • Sequence lengths: min=48, max=48, mean=48.0
   • Missing values: 0 / 590,160 (0.0%)

📋 Sample Sequence (Patient 1):
    alb    alt   ammo  aptt   ast    be   bun  ccr   cl     cr   crp   esr    fib    gfr  gluc     hb   hco3
0  28.0  100.0  154.0  13.4  27.0  35.0  1.03  3.1  1.2  138.0  38.0  7.46  146.0  433.0  12.3  100.0  15.16
1  28.0  100.0  154.0  13.4  27.0  35.0  1.03  3.1  1.2  141.0  38.0  7.46  146.0  433.0  12.3  1

In [14]:
integrated_clinical

,caseid,subjectid,casestart,caseend,anestart,aneend,opstart,opend,adm,dis,...,cr_count,cr_mean,cr_std,cr_min,cr_max,bun_count,bun_mean,bun_std,bun_min,bun_max
0,1,5955,0,11542,-552,10848.0,1668,10368,-236220,627780,...,4.0,0.815,0.094,0.72,0.91,4.0,11.750,2.217,10.0,15.0
1,2,2487,0,15741,-1039,14921.0,1721,14621,-221160,1506840,...,6.0,0.843,0.127,0.71,1.02,6.0,10.833,4.446,6.0,18.0
2,3,2861,0,4394,-590,4210.0,1090,3010,-218640,40560,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1903,0,20990,-778,20222.0,2522,17822,-201120,576480,...,5.0,0.760,0.091,0.66,0.87,5.0,10.000,2.000,8.0,13.0
4,5,4416,0,21531,-1009,22391.0,2591,20291,-67560,3734040,...,21.0,2.945,0.849,1.38,4.43,21.0,35.857,9.624,19.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6383,6384,5583,0,15248,-260,15640.0,2140,14140,-215340,648660,...,4.0,0.945,0.095,0.85,1.07,4.0,8.750,2.500,6.0,12.0
6384,6385,2278,0,20643,-544,20996.0,2396,19496,-225600,1675200,...,10.0,0.876,0.105,0.77,1.14,10.0,15.500,2.369,12.0,19.0
6385,6386,4045,0,19451,-667,19133.0,3533,18233,-200460,836340,...,2.0,0.600,0.028,0.58,0.62,2.0,9.000,1.414,8.0,10.0
6386,6387,5230,0,12025,-550,12830.0,1730,11030,-227760,377040,...,4.0,0.568,0.071,0.49,0.65,4.0,4.500,2.082,2.0,7.0


In [17]:
sample_seq.shape

(48, 17)

In [12]:
# =========================================
# 11. Integration Summary and Next Steps
# =========================================

print("📋 INTEGRATION SUMMARY AND NEXT STEPS")
print("=" * 70)

print("🎯 CLINICAL DATA INTEGRATION COMPLETED!")
print("─" * 50)

print("✅ Successfully Completed:")
print("   1. 📊 Dataset Analysis and Exploration")
print("      • Analyzed all 5 CSV files with comprehensive statistics")
print("      • Identified data quality issues and missing value patterns")
print("      • Created visualizations and quality assessments")

print("\n   2. 🔗 Clinical Data Integration")
print("      • Merged clinical_data.csv with lab_data.csv using caseid")
print("      • Created lab parameter mappings for interpretation")
print("      • Generated integrated clinical dataset with summary statistics")

print("\n   3. ⏰ Time-Series Data Preparation")
print("      • Created time-series sequences for deep learning")
print("      • Prepared data for transformer model training")
print("      • Implemented proper missing value handling")

print(f"\n📊 Final Dataset Summary:")
print(f"   • Integrated Clinical Dataset: {integrated_clinical.shape[0]:,} patients × {integrated_clinical.shape[1]} features")
print(f"   • Time-Series Sequences: {len(sequences)} sequences ready for modeling")
print(f"   • Lab Parameter Coverage: {len(common_lab_names)} parameters with definitions")
print(f"   • Data Integration Coverage: {len(common_caseids)/len(clinical_caseids)*100:.1f}% of patients")

print(f"\n💾 Files Created:")
print(f"   • integrated_clinical_data.csv: Complete integrated dataset")
print(f"   • Time-series sequences: In-memory arrays ready for training")

print(f"\n🚀 NEXT STEPS FOR ANALYSIS:")

print(f"\n   1. 🔬 Missing Value Imputation:")
print(f"      • Apply time-series imputation methods to sequences")
print(f"      • Use forward-fill, interpolation, or advanced methods")
print(f"      • Validate imputation quality using medical knowledge")

print(f"\n   2. 📈 Feature Engineering:")
print(f"      • Extract temporal features (trends, variability, patterns)")
print(f"      • Create clinical ratios and derived scores")
print(f"      • Normalize values using parameter reference ranges")

print(f"\n   3. 🤖 Deep Learning Model Development:")
print(f"      • Train transformer models on time-series sequences")
print(f"      • Implement attention mechanisms for temporal patterns")
print(f"      • Use mortality prediction as target variable")

print(f"\n   4. 📊 Model Evaluation:")
print(f"      • Split data into train/validation/test sets")
print(f"      • Implement cross-validation for robust evaluation")
print(f"      • Use clinical metrics (AUROC, sensitivity, specificity)")

print(f"\n   5. 🔍 Clinical Interpretation:")
print(f"      • Analyze attention weights to identify important time points")
print(f"      • Extract interpretable features for clinical decision support")
print(f"      • Validate findings with medical domain experts")

print(f"\n   6. 📚 Advanced Analysis:")
print(f"      • Implement multi-task learning (mortality, LOS, complications)")
print(f"      • Explore patient similarity and clustering")
print(f"      • Develop real-time prediction systems")

print(f"\n💡 Key Insights from Integration:")
print(f"   • Lab data provides rich temporal information for {len(eligible_patients):,} patients")
print(f"   • {len(common_lab_names)} lab parameters are well-defined and interpretable")
print(f"   • Missing value patterns suggest need for sophisticated imputation")
print(f"   • Time-series structure is suitable for transformer architectures")

print(f"\n🎯 Ready for Advanced Machine Learning!")
print(f"   • Integrated dataset supports both tabular and sequence modeling")
print(f"   • Parameter mappings enable clinical interpretation")
print(f"   • Time-series sequences ready for deep learning pipelines")

print(f"\n" + "="*70)
print(f"🏥 VitalDB Clinical Integration Complete!")
print(f"📊 Ready for mortality prediction and clinical decision support!")


📋 INTEGRATION SUMMARY AND NEXT STEPS
🎯 CLINICAL DATA INTEGRATION COMPLETED!
──────────────────────────────────────────────────
✅ Successfully Completed:
   1. 📊 Dataset Analysis and Exploration
      • Analyzed all 5 CSV files with comprehensive statistics
      • Identified data quality issues and missing value patterns
      • Created visualizations and quality assessments

   2. 🔗 Clinical Data Integration
      • Merged clinical_data.csv with lab_data.csv using caseid
      • Created lab parameter mappings for interpretation
      • Generated integrated clinical dataset with summary statistics

   3. ⏰ Time-Series Data Preparation
      • Created time-series sequences for deep learning
      • Prepared data for transformer model training
      • Implemented proper missing value handling

📊 Final Dataset Summary:
   • Integrated Clinical Dataset: 6,388 patients × 132 features
   • Time-Series Sequences: 543 sequences ready for modeling
   • Lab Parameter Coverage: 33 parameters with